Install modules

In [ ]:
!pip install canvasapi
from canvasapi import Canvas
import os
import requests
from canvasapi.exceptions import CanvasException

Get User Token

In [ ]:
# get url
from google.colab import userdata

# get user token
token = userdata.get('TOKEN')

In [ ]:
#@title My Form
url = "https://umich.beta.instructure.com" #@param {type:"string"}
course_id = 0 #@param {type:"number"}
source_assignment_id = 0 #@param {type:"number"}
target_assignment_id = 0 #@param {type:"number"}

Get Values: Check Course ID, Source ID, and Target ID before executing

In [ ]:
#defined parameters
print('Check the parameters below before running the next step.')

print(f'URL: {url}')
print(f'Course ID: {course_id}')
print(f'Source Assignment ID: {source_assignment_id}')
print(f'Target Assignment ID: {target_assignment_id}')

In [ ]:

# Initialize a new Canvas object
canvas = Canvas(url, token)

# Get source course, assignment, and submission
source_course = canvas.get_course(course_id)
source_assignment = source_course.get_assignment(source_assignment_id)
print(source_assignment.name)

In [ ]:

def create_submission_for_user(user_id, file_ids):
  """
    Submits files for a specific user to a target assignment.

    Parameters:
    user_id (str): The ID of the user for whom the submission is being made.
    file_ids (list): A list of file IDs to be submitted.

    Returns:
    None

    Raises:
    requests.exceptions.RequestException: If there is an error during the submission process.

    Example:
    create_submission_for_user('12345', [67890, 12345])
    """
  submission_url = f"{url}/api/v1/courses/{course_id}/assignments/{target_assignment_id}/submissions"
  headers = {
      'Authorization': f'Bearer {token}',
      'Content-Type': 'application/json'
  }
  data = {
      'submission': {
          'submission_type': 'online_upload',
          'file_ids': file_ids
      }
  }
  params = {'as_user_id': user_id}

  try:
      response = requests.post(submission_url, headers=headers, json=data, params=params)
      response.raise_for_status()
      print(f'Successfully submitted for student {user_id}:')
  except requests.exceptions.RequestException as err:
      print(f'Error submitting files for student {user_id}: {err}')

In [ ]:
# get all submission for source_assignment
submissions = source_assignment.get_submissions()

for submission in submissions:
  user_id = submission.user_id

  # get all attachments for the original submission
  file_ids=[]
  for attachment in submission.attachments:
    file_id = attachment.id
    file_ids.append(file_id)
  print(f"user_id = {user_id} file_ids={file_ids}")

  # create submission for user when there is at least one attachment
  if len(file_ids) > 0:
    create_submission_for_user(user_id, file_ids)
